In [20]:
import pandas as pd
import numpy as np
import torch
import tqdm

In [14]:
device = torch.device('cpu')
df_test_text = pd.read_csv("test.csv")
df_test_labels = pd.read_csv("test_labels.csv")[['id', 'toxic']]
df_test = pd.merge(df_test_text, df_test_labels, on='id')
df_test.head(10)

df_test['text'] = df_test['comment_text']
df_test.drop(['id', 'comment_text'], inplace=True, axis=1)
df_test.head(20)

df_test = df_test.drop(df_test[df_test['toxic'] == -1].index)
df_test

,toxic,text
5,0,Thank you for understanding. I think very high...
7,0,:Dear god this site is horrible.
11,0,"""::: Somebody will invariably try to add Relig..."
13,0,""" \n\n It says it right there that it IS a typ..."
14,0,""" \n\n == Before adding a new product to the l..."
...,...,...
153150,0,":Jerome, I see you never got around to this…! ..."
153151,0,==Lucky bastard== \n http://wikimediafoundatio...
153154,0,==shame on you all!!!== \n\n You want to speak...
153155,1,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...


In [17]:
def get_pred(text, classifier, tokenizer):
    text_token = tokenizer("hello world", return_tensors="pt")
    output = classifier(**text_token)
    prediction = torch.sigmoid(output.logits)*100
    prediction = prediction.detach().numpy().tolist()[0]
    return prediction[0]


In [31]:
y_pred = []
y_true = []
classifier = AutoModelForSequenceClassification.from_pretrained("sachiniyer/tweet_toxicity")
tokenizer = AutoTokenizer.from_pretrained("sachiniyer/tweet_toxicity")
for d in df_test.itertuples():
    y_pred.append(round(get_pred(d.text, classifier, tokenizer)))
    y_true.append(d.toxic)
    if d.Index % 1000 == 0: print(d.Index)

from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_true)

0.9058669774999117